# Injection benchmark

Inject in a box a maxwellian at a given temperature and density. The box is initially empty. It should converge to the right density and have the right temperature.

To start with, the dsmc-collisions are deactivated and only the density convergence is studied. Then, temperature convergence will be studied.

All cases :
- 0D-box (1 cell only) - size : $1e-1 \times 1e-1 \times 1e-2$.
- Iodine at density $3.2\times 10^{19}$ $m^{-3}$
- Temperature is 300 K
- injection wall is also an exiting wall (any particle colliding with the wall exits the system.
- injection is performed by the left boundary.
- time step : usually the mean free path of a particle for the aformentioned values is $4e-2$ $m$ (that why we chose $dx = dy = dz = 1e-2$). With a mean speed of $200$ $m/s$, it yields at time step of roughly $5 \times 10^{-5}$ $s$ (a fourth of the mean free path). We eventually take  $1 \times 10^{-5}$ $s$. 

Case 1: 
- the three other boundaries have specular reflection

Case 2:
- the three other boundaries have diffusive reflection at 300 K

Conclusion :
For the first two cases, we see that the specular reflections causes an issue with the density (ot so much with thermalization). 
We tend to be slightly below the right density for the specular reflections (5% for the parameters). We are really on point for the diffusive though.
Increasing the number of cells, increased the number of collisions with particles for particles before leaving the system and we got closer to the target density.

Case 3: Couette
- the three other boundaries have diffusive reflection at 300 K
- A drift in the 'x' direction is added for the top  boundary : in this case, a gradient will be present and there should be enough cells to represent it.

In [1]:
%matplotlib widget 
# global import
import numpy as np
import pandas as pd

# plotting
import matplotlib.pyplot as plt

# local import
import lppydsmc as ld

In [2]:
simulation = False

## Case 1

In [3]:
path_injection_1 = "/home/calot/Documents/projets/lppydsmc/benchmarks/config_files/injection_1.ini"

In [4]:
# launching simulations
if(simulation):
    ld.main(path_injection_1, save = True);

### Analyzing results

In [5]:
results_path = "/home/calot/Documents/projets/lppydsmc/benchmarks/results/injection_1_2/monitoring.h5" # seems ok ...
store = pd.HDFStore(results_path)
print(store.keys())

['/dsmc_collisions', '/dsmc_tracking', '/fluxes', '/out_particles', '/particles', '/wall_collisions']


In [6]:
dt = 1e-5 
dx, dy, dz = 1e-2, 1e-2, 1e-2
nb_cells = 10*10
particles_weight = 320000000000
target_density = 3.2e19

In [7]:
df_particles = store['particles']

#### Density

In [8]:
evo_nb_particles = df_particles['x'].groupby(df_particles.index).count()
evo_nb_particles = evo_nb_particles/(dx*dy*dz*nb_cells)*particles_weight
mean_density_steady_flow = np.mean(evo_nb_particles.values[8000:])

In [9]:
fig, ax = plt.subplots()
print('Mean value of last 300 iterations : {:.3e} m^-3'.format(mean_density_steady_flow))
print('Relative difference to target : {:.3} %'.format(100*abs(target_density-mean_density_steady_flow)/target_density))
print('Min / max value : {:.3e} / {:.3e} m^-3'.format(np.min(evo_nb_particles), np.max(evo_nb_particles)))
ax.plot(evo_nb_particles.index*dt, evo_nb_particles.values);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Mean value of last 300 iterations : 3.196e+19 m^-3
Relative difference to target : 0.123 %
Min / max value : 1.760e+17 / 3.226e+19 m^-3


#### Thermal equilibrium

In [10]:
mass = 2.16e-25

In [11]:
df_particles['v2'] = df_particles['vx']*df_particles['vx']+df_particles['vy']*df_particles['vy']+df_particles['vz']*df_particles['vz']
df_particles['v'] = np.sqrt(df_particles['v2'])

In [12]:
frames = 0.8 # taking the last 80% of the simulation
max_frame = np.max(df_particles.index)
df_partial = df_particles.loc[df_particles.index > frames*max_frame]

In [13]:
bins = 100

fig, ax = plt.subplots(2,2, constrained_layout = True)
ax[0,0].hist(df_partial['vx'], bins = bins, color = 'r')
ax[0,1].hist(df_partial['vy'], bins = bins, color = 'g')
ax[1,0].hist(df_partial['vz'], bins = bins, color = 'b')
ax[1,1].hist(df_partial['v'], bins = bins, color = 'k');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [24]:
mean = df_partial.mean()

In [30]:
v = df_partial['v']
drift = np.array([mean['vx'],mean['vy'],mean['vz']])
drift_norm = np.linalg.norm(drift)
kb = ld.utils.physics.BOLTZMAN_CONSTANT
print('Drift : {} m/s'.format(drift))
print("Local temperature : {:.3e} K".format((np.mean(v*v)-drift_norm*drift_norm)*mass/(kb*3)))
print('Standard deviation on the gaussian : {:.3e} m/s'.format(np.std(df_partial['vx'])))

Drift : [-0.9409625   0.0495034   1.00913292] m/s
Local temperature : 2.986e+02 K
Standard deviation on the gaussian : 1.379e+02 m/s


#### Conclusion

- Density : OK
- Maxwell-Boltzmann distribution at 300 K : OK

## Case 2

In [3]:
path_injection_2 = "/home/calot/Documents/projets/lppydsmc/benchmarks/config_files/injection_2.ini"

In [4]:
# launching simulations
if(simulation):
    ld.main(path_injection_2, save = True);

### Analyzing results

In [3]:
results_path = "/home/calot/Documents/projets/lppydsmc/benchmarks/results/injection_2_2/monitoring.h5" # seems ok ...
store = pd.HDFStore(results_path)
print(store.keys())

['/dsmc_collisions', '/dsmc_tracking', '/fluxes', '/out_particles', '/particles', '/wall_collisions']


In [4]:
dt = 1e-5 
dx, dy, dz = 1e-2, 1e-2, 1e-2
nb_cells = 1*1
particles_weight = 3200000000
target_density = 3.2e19

In [5]:
df_particles = store['particles']

#### Density

In [6]:
steady_frame = 2000
evo_nb_particles = df_particles['x'].groupby(df_particles.index).count()
evo_nb_particles = evo_nb_particles/(dx*dy*dz*nb_cells)*particles_weight
mean_density_steady_flow = np.mean(evo_nb_particles.loc[evo_nb_particles.index>steady_frame])

In [30]:
fig, ax = plt.subplots(constrained_layout = True)
print('Mean value of last 300 iterations : {:.3e} m^-3'.format(mean_density_steady_flow))
print('Relative difference to target : {:.3} %'.format(100*abs(target_density-mean_density_steady_flow)/target_density))
print('Min / max value : {:.3e} / {:.3e} m^-3'.format(np.min(evo_nb_particles), np.max(evo_nb_particles)))
ax.plot(evo_nb_particles.index*dt, (target_density-evo_nb_particles.values)/target_density, label = r'$\frac{|n - n(t)|}{n}$');
ax.set_yscale('log');
ax.set_xlabel('t (s)',fontsize = 12);
# ax.set_ylabel(r'$\frac{|n - n(t)|}{n}$', rotation = 'horizontal', fontsize  = 12, y = 0.8)
ax.legend(loc='upper right', fontsize = 15)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Mean value of last 300 iterations : 3.112e+19 m^-3
Relative difference to target : 2.75 %
Min / max value : 1.757e+18 / 3.174e+19 m^-3


#### Thermal equilibrium

In [8]:
mass = 2.16e-25

In [9]:
from analysis import validate_maxwellian_distribution

In [10]:
fig, ax = plt.subplots(constrained_layout=True)
validate_maxwellian_distribution(ax, df_particles, steady_frame, mass, temperature = 300)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:xlabel='Translational energy (eV)', ylabel='Probability density $f(E_t, T)$'>

In [13]:
df_particles['v2'] = df_particles['vx']*df_particles['vx']+df_particles['vy']*df_particles['vy']+df_particles['vz']*df_particles['vz']
df_particles['v'] = np.sqrt(df_particles['v2'])

In [14]:
frames = 0.3 # taking the last 80% of the simulation
max_frame = np.max(df_particles.index)
df_partial = df_particles.loc[df_particles.index > frames*max_frame]

In [15]:
bins = 100

fig, ax = plt.subplots(2,2, constrained_layout = True)
ax[0,0].hist(df_partial['vx'], bins = bins, color = 'r', density = True)
ax[0,1].hist(df_partial['vy'], bins = bins, color = 'g', density = True)
ax[1,0].hist(df_partial['vz'], bins = bins, color = 'b', density = True)
ax[1,1].hist(df_partial['v'], bins = bins, color = 'k', density = True);

ax[0,0].set_xlabel('vx (m/s)')
ax[0,1].set_xlabel('vy (m/s)')
ax[1,0].set_xlabel('vz (m/s)')
ax[1,1].set_xlabel('v (m/s)');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
mean = df_partial.mean()

In [17]:
v = df_partial['v']
drift = np.array([mean['vx'],mean['vy'],mean['vz']])
drift_norm = np.linalg.norm(drift)
kb = ld.utils.physics.BOLTZMAN_CONSTANT
print('Drift : {} m/s'.format(drift))
print("Local temperature : {:.3e} K".format((np.mean(v*v)-drift_norm*drift_norm)*mass/(kb*3)))
print('Standard deviation on the gaussian : {:.3e} m/s'.format(np.std(df_partial['vx'])))

Drift : [-9.82169486 -0.22682544 -0.09679699] m/s
Local temperature : 2.957e+02 K
Standard deviation on the gaussian : 1.353e+02 m/s


#### Conclusion

- Density : OK
- Maxwell-Boltzmann distribution at 300 K : OK

## Case 3 : Couette but with injection

The objective is to make a comparison with the previous tests in order to prepare for the bigger tests with much more cells and no injection (initialization instead). 
It is interesting to see what happens when you simply change two conditions :
- drift for the top wall
- specular reflection for the top wall

Of course, it is a very useless simulation for benchmarking but may be it can help understanding what happens next.

------

What we see immediately is that the density in the system increases compared to the previous simulation : indeed this is because the top wall is biased, it tends to make particles go to the right whereas the left wall is the "out-wall". Here it is roughly a 8 to 10% increase.

This is the same phenomena than between the first case and second one: adding diffusivity randomize the velocities whereas when we inject with only specular reflections, the particles going straight (vx >> vy) tend to leave the system immediately after the first reflection.

In [26]:
path_injection_3 = "/home/calot/Documents/projets/lppydsmc/benchmarks/config_files/injection_3.ini"

In [28]:
# launching simulations
ld.main(path_injection_3, save = True);

|    it    |   INIT   |  INJECT  |   DEL    | C. WALLS | C. PARTS |  REACT   | EXEC TIME (s) |
----------------------------------------------------------------------------------------------
|    0     |    0     |    0     |    0     |    0     |    0     |    NA    |       0       |
|   100    |   4838   |    55    |    22    |   100    |   123    |    NA    |   1.418e-02   |
|   200    |   7770   |    55    |    36    |   158    |   207    |    NA    |   2.475e-02   |
|   300    |   9240   |    56    |    42    |   168    |   245    |    NA    |   3.307e-02   |
|   400    |  10009   |    55    |    60    |   194    |   243    |    NA    |   4.381e-02   |
|   500    |  10451   |    55    |    48    |   171    |   265    |    NA    |   5.071e-02   |
|   600    |  10632   |    55    |    43    |   173    |   280    |    NA    |   4.851e-02   |
|   700    |  10788   |    55    |    56    |   171    |   263    |    NA    |   4.719e-02   |
|   800    |  10785   |    56    |    61    |   19

### Analyzing results

In [29]:
results_path = "/home/calot/Documents/projets/lppydsmc/benchmarks/results/injection_3_1/monitoring.h5" # seems ok ...
store = pd.HDFStore(results_path)
print(store.keys())

['/dsmc_collisions', '/dsmc_tracking', '/fluxes', '/out_particles', '/particles', '/wall_collisions']


In [30]:
dt = 1e-5 
dx, dy, dz = 1e-2, 1e-2, 1e-2
nb_cells = 10*10
particles_weight = 320000000000
target_density = 3.2e19

In [31]:
df_particles = store['particles']

#### Density

In [32]:
evo_nb_particles = df_particles['x'].groupby(df_particles.index).count()
evo_nb_particles = evo_nb_particles/(dx*dy*dz*nb_cells)*particles_weight
mean_density_steady_flow = np.mean(evo_nb_particles.values[8000:])

In [33]:
fig, ax = plt.subplots()
print('Mean value of last 300 iterations : {:.3e} m^-3'.format(mean_density_steady_flow))
print('Relative difference to target : {:.3} %'.format(100*abs(target_density-mean_density_steady_flow)/target_density))
print('Min / max value : {:.3e} / {:.3e} m^-3'.format(np.min(evo_nb_particles), np.max(evo_nb_particles)))
ax.plot(evo_nb_particles.index*dt, evo_nb_particles.values);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Mean value of last 300 iterations : 3.473e+19 m^-3
Relative difference to target : 8.54 %
Min / max value : 1.760e+17 / 3.529e+19 m^-3


#### Thermal equilibrium

In [34]:
mass = 2.16e-25

In [35]:
df_particles['v2'] = df_particles['vx']*df_particles['vx']+df_particles['vy']*df_particles['vy']+df_particles['vz']*df_particles['vz']
df_particles['v'] = np.sqrt(df_particles['v2'])

In [36]:
frames = 0.8 # taking the last 80% of the simulation
max_frame = np.max(df_particles.index)
df_partial = df_particles.loc[df_particles.index > frames*max_frame]

In [69]:
bins = 100

fig, ax = plt.subplots(2,2, constrained_layout = True)
ax[0,0].hist(df_partial['vx'], bins = bins, color = 'r')
ax[0,1].hist(df_partial['vy'], bins = bins, color = 'g')
ax[1,0].hist(df_partial['vz'], bins = bins, color = 'b')
ax[1,1].hist(df_partial['v'], bins = bins, color = 'k');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [38]:
mean = df_partial.mean()

In [39]:
v = df_partial['v']
drift = np.array([mean['vx'],mean['vy'],mean['vz']])
drift_norm = np.linalg.norm(drift)
kb = ld.utils.physics.BOLTZMAN_CONSTANT
print('Drift : {} m/s'.format(drift))
print("Local temperature : {:.3e} K".format((np.mean(v*v)-drift_norm*drift_norm)*mass/(kb*3)))
print('Standard deviation on the gaussian : {:.3e} m/s'.format(np.std(df_partial['vx'])))

Drift : [-0.82173833 -1.66189331 -0.29664523] m/s
Local temperature : 3.169e+02 K
Standard deviation on the gaussian : 1.448e+02 m/s


#### Conclusion
As expected we have different things although almost no drift is to be seen (and not one that would be expected). => we have gradients.

We see that the temperature is higher meaning the system is heaten by the top wall.

In [40]:
import seaborn as sns

In [51]:
fig, ax = plt.subplots()
sns.histplot(data = df_partial, x = 'x', y = 'vx', cbar = True, bins = 50, ax = ax)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:xlabel='x', ylabel='vx'>

In [50]:
fig, ax = plt.subplots()
sns.histplot(data = df_partial, x = 'vx', y = 'y', cbar = True, bins = 50, ax = ax)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:xlabel='vx', ylabel='y'>

In [53]:
df_particles['y_int'] = (df_particles['y']/dy).astype(int) 

In [56]:
results = df_partial.groupby('y_int').mean()['vx']

In [59]:
fig, ax = plt.subplots()
ax.plot(results.values, results.index*dy);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [66]:
df_particles['x_int'] = (df_particles['x']/dx).astype(int) 

In [67]:
results = df_partial.groupby('x_int').mean()['vx']
fig, ax = plt.subplots()
ax.plot(results.index*dx, results.values);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …